In [ ]:
import csv
import pandas as pd
import pickle
import nltk
import matplotlib.pyplot as plt

# Manual Intervention
In dataOCM/02_LDA/<span></span>LDA_Runs/01_All, there are models trained on different number of topics

Copy the timestamp folder belonging to LDA Model for dataset1 and dataset2 to dataOCM/02_LDA/<span></span>LDA_Runs/03_Selected

Supply the folder name and no. of topics in below mentioned if condition

In [ ]:
# Regex tokenization
tokenizer = nltk.RegexpTokenizer(r"\w+")

# For counting words in review
def wordsCounter(review):
    doc = review
    doc = tokenizer.tokenize(doc)
    wordsCount = len(doc)
    return wordsCount

# Dataset 1/2

In [ ]:
# Dataset switch
dataset = 1
if dataset == 1:
    LDAModelTimeStamp = '07092020_113759' # Dataset1
    noOfTopicsFolder = 'noOfTopics_18'
    topicsToEliminate = []
    topicsLabelsDict = {'topic0': ['bad companies', 'BCP'],
                        'topic1': ['job pressure', 'JBP'],
                        'topic2': ['work atmosphere - interesting tasks', 'WAI'],
                        'topic3': ['cohesion among colleagues', 'CAC'],
                        'topic4': ['treatment of colleagues', 'TOC'],
                        'topic5': ['equal rights', 'EQR'],
                        'topic6': ['work life balance', 'WLB'],
                        'topic7': ['treatment of leader', 'TOL'],
                        'topic8': ['communication in the organization', 'CMO'],
                        'topic9': ['flexible work hours', 'FWH'],
                        'topic10': ['circumstances at work - overtime', 'CWO'],
                        'topic11': ['age diversity', 'AGD'],
                        'topic12': ['vacation', 'VAC'],
                        'topic13': ['work from home', 'WFH'],
                        'topic14': ['good management', 'GMN'],
                        'topic15': ['work hours', 'WHR'],
                        'topic16': ['work atmosphere', 'WAT'],
                        'topic17': ['work atmosphere for new colleagues', 'WAN']}
elif dataset == 2:
    LDAModelTimeStamp = '08092020_083631'  # Dataset2
    noOfTopicsFolder = 'noOfTopics_23'
    topicsToEliminate = ['topic12', 'topic20', 'topic22']
    topicsLabelsDict = {'topic0': ['relationships with colleagues and bosses', 'RCB'],
                        'topic1': ['satisfaction with work and colleagues', 'SWC'],
                        'topic2': ['work hours', 'WHR'],
                        'topic3': ['great education', 'GED'],
                        'topic4': ['gender diversity', 'GND'],
                        'topic5': ['cultural diversity', 'CLD'],
                        'topic6': ['overtime, job pressure', 'OJP'],
                        'topic7': ['work life balance', 'WLB'],
                        'topic8': ['women empowerment', 'WEM'],
                        'topic9': ['appreciation of experience', 'APE'],
                        'topic10': ['age diversity', 'AGD'],
                        'topic11': ['cohesion among colleagues', 'CAC'],
                        'topic13': ['cohesion among new colleagues', 'CNC'],
                        'topic14': ['fun environment', 'FUN'],
                        'topic15': ['young team', 'YNT'],
                        'topic16': ['communication in the organisation', 'CMO'],
                        'topic17': ['work from home', 'WFH'],
                        'topic18': ['bad salary', 'BDS'],
                        'topic19': ['tasks at work', 'TAW'],
                        'topic21': ['female leaders', 'FML']}

# LDA Fitted data target location

In [ ]:
dir = 'dataOCM/02_LDA/LDA_Runs/03_Selected/' + LDAModelTimeStamp
topicsDocumentsDistribution = pickle.load(open(dir + '/' + noOfTopicsFolder + '/' +  'LDA_04_Fitting_fittedData_' + LDAModelTimeStamp + '.data', 'rb'))
tDocDist_df = pd.DataFrame(topicsDocumentsDistribution)
new_columns = tDocDist_df.loc[0]
tDocDist_df.columns = new_columns

# Counting number of topics

In [ ]:
noOfTopics = 0
for name in new_columns:
    if 'topic' in name:
        noOfTopics +=1

# Size of Dataframe

In [ ]:
tDocDist_df = tDocDist_df[1:tDocDist_df.size]
seriesLen = tDocDist_df['RvComment'].size
print('size of dataframe is :', seriesLen)

# Counting number of words for each review in dataframe

In [ ]:
tDocDist_df['noOfWords'] = tDocDist_df.apply(lambda x: wordsCounter(x['RvComment']), axis=1)

# Reviews Descriptive Statistics export to csv

In [ ]:
dfRecom = tDocDist_df[tDocDist_df['RverRecom']=='Empfohlen']
dfNotRecom = tDocDist_df[tDocDist_df['RverRecom']=='Nicht empfohlen']
tDocDist_df['Org'].value_counts().to_csv('dataOCM/02_LDA/Descriptive_Statistics_Reviews/DescStats_Dataset_' + str(dataset) + '_AllReviews_CompanyWise.csv')
dfRecom['Org'].value_counts().to_csv('dataOCM/02_LDA/Descriptive_Statistics_Reviews/DescStats_Dataset_' + str(dataset) + '_RecomReviews_CompanyWise.csv')
dfNotRecom['Org'].value_counts().to_csv('dataOCM/02_LDA/Descriptive_Statistics_Reviews/DescStats_Dataset_' + str(dataset) + '_NotRecomReviews_CompanyWise.csv')
tDocDist_df['noOfWords'].to_csv('dataOCM/02_LDA/Descriptive_Statistics_Reviews/DescStats_Dataset_' + str(dataset) + '_WordsInReviews.csv')

# Output: Cumulative Topics Proportions csv

In [ ]:
csvFileNameOut = 'dataOCM/02_LDA/Cumulative_Topics_Proportions/RQ1_CumulativeTopicsProportions_Dataset_' + dataset + '.csv'
csvFileOut = open(csvFileNameOut, "w", newline='', encoding='utf-8')
csv_out = csv.writer(csvFileOut, delimiter='|')

# Populating 2D list

In [ ]:
topicsProportions = []
for t in range(noOfTopics):
    topicsProportions_temp = []
    topicsProportions_temp.append('topic' + str(t))
    topicsProportions_temp.append(sum(tDocDist_df['topic' + str(t)]))
    topicsProportions_temp.append(0)
    topicsProportions.append(topicsProportions_temp)

# Calculating sum of all topics for normalization

In [ ]:
sumOfAllinDenom = sum(topicsProportions[c][1] for c in range(len(topicsProportions)) if topicsProportions[c][0] not in topicsToEliminate)

# Populating 2D list with normalized values

In [ ]:
for t in range(noOfTopics):
    if (topicsProportions[t][0] not in topicsToEliminate):
        topicsProportions[t][2] = topicsProportions[t][1] / sumOfAllinDenom
        csv_out.writerow(topicsProportions[t])
    else:
        topicsProportions[t][2] = 'topic eliminated'
        csv_out.writerow(topicsProportions[t])

# Saving the Fitted Dataset as dataframe

In [ ]:
with open('dataOCM/02_LDA/Post_LDA_Dataset_' + dataset + '.df', 'wb') as f:
    pickle.dump(tDocDist_df, f)

# Plotting Bar Chart

In [ ]:
xAxis = [topicsLabelsDict[i[0]][1] for i in topicsProportions]
yAxis = [round(i[2]*100,2) for i in topicsProportions]
plt.rcParams.update({'font.size': 8})
if dataset == 1:
    fig = plt.figure(figsize=(9 / 2.54, 9 / 2.54))  # 10.45 - 2*0.4322 #dt1
elif dataset == 2:
    fig = plt.figure(figsize=(9/2.54,10.35/2.54)) #dt2
ax = fig.add_axes([0.12,0.12,0.86,0.87])
for i, v in enumerate(yAxis):
    ax.text(v + 0.15, i - 0.2, str(v), color='#185aa9')
ax.set_xlabel('Cumulative Topics Proportions %')
ax.set_xlim(0,14)
# ax.set_title('Cumulative Topics Proportions - Dataset1')
# ax.xaxis.grid(True)
# ax.grid(True)
# ax.grid(color='b', ls = '-.', lw = 0.25)
colors = [(200/255,200/255,200/255) for i in range(len(yAxis))]
colors[len(colors)-1] = (122/255,195/255,106/255)
colors[len(colors)-2] = (122/255,195/255,106/255)
colors[0] = (241/255,90/255,96/255)
colors[1] = (241/255,90/255,96/255)
ax.barh(xAxis,yAxis, color = colors, height = 0.5)
plt.savefig('dataOCM/02_LDA/Cumulative_Topics_Proportions/Exp_1_Cumulative_Topics_Proportions_Dataset' + dataset + '.png')
plt.show()